In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import glob

import blimpy
from blimpy import Waterfall
from blimpy import utils
from blimpy import plotting

import turbo_seti.find_doppler.seti_event as turbo
import turbo_seti.find_event as find_event
from turbo_seti.find_doppler.find_doppler import FindDoppler
from turbo_seti.find_event.find_event_pipeline import find_event_pipeline
from turbo_seti.find_event.plot_event_pipeline import plot_event_pipeline
import pandas as pd
%matplotlib inline

print("All packages imported!")

All packages imported!


In [2]:
def grab_file_path(data_dir,file_type,node_number):
    '''
    returns h5 and dat file path from given directory, ordered correctly
    '''
    
    ## h5 list
    data_list = []
    for dirname, _, filenames in os.walk(data_dir):
        for filename in filenames:
            if filename[-len(file_type):] == file_type and node_number in filename:
                data_list.append(filename)
    
    data_list = sorted(data_list, key=lambda x: (x,x.split('_')[5]))

    print(data_list)
    
    data_path = os.path.join(data_dir, file_type[1:]+'_list.lst')
    with open(data_path, 'w') as f:
        for path in data_list:
            f.write(data_dir+path + '\n')

    #You don't have to print, but it's a good way to check that your list is in the correct order:
    with open(data_path, 'r') as f:
        print(f.read())
    f.close()
    
    return data_path

In [3]:
def files_plotter(files,f_start,f_end,show_plot):
    '''
    Grabs the data of the waterfall object segment you are interested in
    '''

    this_cadence_data = []
    this_cadence_extents = []
    if show_plot == True:
        fig, axs = plt.subplots(len(files),figsize=(10, 1.5*len(files)))
    for file_number,file in enumerate(files):

    # print('getting data',file)
        obs = Waterfall(file,f_start=f_start,f_stop=f_end)
        # data = obs.data
        plot_f, plot_data = obs.grab_data(f_start=f_start,f_stop=f_end)
        plot_data = plot_data.astype('float32')
        # print('got data')
    
        #
        MAX_PLT_POINTS      = 65536                  # Max number of points in matplotlib plot
        MAX_IMSHOW_POINTS   = (8192, 4096)           # Max number of points in imshow plot
    
    
        dec_fac_x, dec_fac_y = 1, 1
        if plot_data.shape[0] > MAX_IMSHOW_POINTS[0]:
            dec_fac_x = int(plot_data.shape[0] / MAX_IMSHOW_POINTS[0])
    
        if plot_data.shape[1] > MAX_IMSHOW_POINTS[1]:
            dec_fac_y = int(plot_data.shape[1] / MAX_IMSHOW_POINTS[1])
            
        plot_data = utils.rebin(plot_data, dec_fac_x, dec_fac_y)
        extent = plotting.plot_utils.calc_extent(obs, plot_f=plot_f, plot_t=obs.timestamps, MJD_time=60098.829675925925)
        reverse=False
        if reverse==True:
                plot_data = plot_data[..., ::-1]  # Reverse data
                plot_f = plot_f[::-1]
        new_extent = list(extent)
        new_extent[2] = 0
        new_extent[3] = 292.057776
        new_extents = tuple(new_extent)


        if show_plot == True:
            axs[file_number].imshow(plot_data,
                           aspect='auto',
                           origin='lower',
                           rasterized=True,
                           interpolation='nearest',
                           extent = new_extents,
                           cmap='viridis',
                           )
            axs[file_number].set_xlabel("freq Mhz",fontsize=10)
            axs[file_number].tick_params(axis='x', which='major', labelsize=1)
            axs[file_number].set_ylim( axs[file_number].get_ylim()[::-1])
            axs[file_number].tick_params(axis='y', which='major', labelsize=15)
    
            if file_number == 5:
                axs[file_number].tick_params(axis='x', which='major', labelsize=10)
        this_cadence_data.append(plot_data)
        this_cadence_extents.append(extent)


    if show_plot == True:
        plt.subplots_adjust(hspace=0)
        plt.show()


    return this_cadence_data, this_cadence_extents

In [4]:
def cycle_events(all_events,data_dir,show_plot):
    '''
    Cycles through the hits of a given snr, plots them, and returns the events as np arrays 
    '''
    print('cycling!')
    all_plot_data = []
    all_extents = []
    all_file_names = []
    for i in range(0,len(all_events)):
        file = data_dir + all_events["FileID"][i]
        file = file[:-3]+"h5"
        drift_rate = all_events["DriftRate"][i]
        freq = all_events["Freq"][i]
        f_start = freq - (drift_rate/10**6)*2000
        f_end = freq + (drift_rate/10**6)*2000
    
    
        node = file.split("/")[-1][0:5]
        print(f"================================ {i} -- NODE {node} =============================")
        print("f_start,",f_start,"f_stop:",f_end,"file:",file)
              
        h5_list_path = grab_file_list(data_dir, '.h5',node)
        this_cadence_data = []
        this_cadence_extents = []
        all_file_names.append(h5_list_path)        
        # print(h5_list_path)
        this_cadence_data, this_cadence_extents = files_plotter(h5_list_path,f_start,f_end,show_plot)
        

        all_plot_data.append(this_cadence_data)
        all_extents.append(this_cadence_extents)
    return all_plot_data, all_extents, all_file_names


In [5]:
def grab_file_list(data_dir,file_type,node_number):
    '''
    returns h5 and dat file path from given directory, ordered correctly
    '''
    
    ## h5 list
    data_list = []
    for dirname, _, filenames in os.walk(data_dir):
        for filename in filenames:
            if filename[-len(file_type):] == file_type and node_number in filename:
                data_list.append(data_dir + filename)
                
    data_list = sorted(data_list, key=lambda x: (x,x.split('_')[5]))

    return data_list

In [8]:
def plot_events(snr):
    total_plot_data = []
    total_extents = []
    total_file_names = []
    for galaxy in sample2:
        print("Galaxy: ",galaxy)
        try:
            galaxy_csv = pd.read_csv(f"/datax/scratch/calebp/seticore_testing/snr_testing/{snr}off10on/{galaxy}/master.csv")
            all_plot_data, all_extents, all_file_names = cycle_events(galaxy_csv,f"/datax/scratch/calebp/seticore_testing/snr_testing/{snr}off10on/{galaxy}/",False)
            for i in all_plot_data:
                total_plot_data.append(i)
            for i in all_extents:
                total_extents.append(i)
            for i in all_file_names:
                total_file_names.append(i)
        except:
            print(f"No events for {galaxy}")
    
    return total_plot_data, total_extents, total_file_names


In [7]:
sample2 = ['AND_XXIII']

In [16]:
sample1 = ['MESSIER031', 'MESSIER033', 'MESSIER081', 'MESSIER101', 'MESSIER49', 'MESSIER59', 'MESSIER60','MESSIER84', 'MESSIER86', 'MESSIER87', 'NGC0185', 'NGC0628', 'NGC0672 ', 'NGC1052', 'NGC1172 ', 'NGC1400', 'NGC1407', 'NGC2403','NGC2683', 'NGC2787', 'NGC3193', 'NGC3226', 'NGC3344', 'NGC3379', 'NGC4136', 'NGC4168', 'NGC4239', 'NGC4244', 'NGC4258', 'NGC4318', 'NGC4365', 'NGC4387', 'NGC4434', 'NGC4458', 'NGC4473', 'NGC4478', 'NGC4486B', 'NGC4489', 'NGC4551', 'NGC4559', 'NGC4564', 'NGC4600', 'NGC4618', 'NGC4660', 'NGC4736', 'NGC4826', 'NGC5194', 'NGC5195', 'NGC5322', 'NGC5638', 'NGC5813', 'NGC5831', 'NGC584', 'NGC5845', 'NGC5846', 'NGC596', 'NGC636', 'NGC6503', 'NGC6822', 'NGC6946', 'NGC720', 'NGC7454 ', 'NGC7640', 'NGC821', 'PEGASUS', 'SAG_DIR', 'SEXA', 'SEXB', 'SEXDSPH', 'UGC04879', 'UGCA127', 'UMIN']

In [9]:
all_plot_data, all_extents, all_file_names = plot_events(8)

Galaxy:  AND_XXIII
cycling!
================================ 0 -- NODE blc02 =============================
f_start, 1697.605733728 f_stop: 1697.608958272 file: /datax/scratch/calebp/seticore_testing/snr_testing/8off10on/AND_XXIII/blc02_guppi_59404_21457_And_XXIII_0088.rawspec.0000.h5
================================ 1 -- NODE blc02 =============================
f_start, 1697.606267094 f_stop: 1697.6096549059998 file: /datax/scratch/calebp/seticore_testing/snr_testing/8off10on/AND_XXIII/blc02_guppi_59404_22099_And_XXIII_0090.rawspec.0000.h5
================================ 2 -- NODE blc02 =============================
f_start, 1697.602857954 f_stop: 1697.605960046 file: /datax/scratch/calebp/seticore_testing/snr_testing/8off10on/AND_XXIII/blc02_guppi_59404_21457_And_XXIII_0088.rawspec.0000.h5
================================ 3 -- NODE blc02 =============================
f_start, 1697.6034415440001 f_stop: 1697.606584456 file: /datax/scratch/calebp/seticore_testing/snr_testing/8off10on/A

In [19]:
len(all_plot_data[1][1])

16

In [26]:
np.save("/datax/scratch/calebp/seticore_testing/all_plot_data.np",all_plot_data)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (603, 6, 16) + inhomogeneous part.

In [42]:
import pickle

with open("/datax/scratch/calebp/seticore_testing/all_plot_data.np", "wb") as fp:   #Pickling
    pickle.dump(all_plot_data, fp)

In [43]:
with open("/datax/scratch/calebp/seticore_testing/all_plot_data.np", "rb") as fp:   #Pickling
    b = pickle.load(fp)

In [20]:
import pickle

with open("/datax/scratch/calebp/seticore_testing/anomaly.np", "wb") as fp:   #Pickling
    pickle.dump(all_plot_data, fp)